In [1]:
!pip install transformers
!pip install torch
!pip install datasets
!pip install sentencepiece
!pip install transformers[sentencepiece]
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 13.3 MB/s 
     |████████████████████████████████| 6.6 MB 66.5 MB/s 
     |████████████████████████████████| 101 kB 10.3 MB/s 
     |████████████████████████████████| 596 kB 51.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 13.7 MB/s 
     |████████████████████████████████| 1.1 MB 57.8 MB/s 
     |████████████████████████████████| 212 kB 69.0 MB/s 
     |████████████████████████████████| 140 kB 71.5 MB/s 
     |████████████████████████████████| 127 kB 69.7 MB/s 
     |██████████████████████████

In [2]:
from google.colab import auth, drive
import os
import numpy as np

from transformers import MarianMTModel, MarianTokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, Trainer, TrainingArguments
from datasets import load_metric

import torch


In [3]:
auth.authenticate_user()

In [4]:
drive.mount('/content/drive')
%cd drive/MyDrive/W266/final_project


Mounted at /content/drive
/content/drive/MyDrive/W266/final_project


In [5]:
os.environ["WANDB_DISABLED"]="true"

In [6]:
source_language = 'en'
target_language = 'es'

model_checkpoint = f"Helsinki-NLP/opus-mt-{source_language}-{target_language}"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/783k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/807k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/marian/tokenization_marian.py:198: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Downloading:   0%|          | 0.00/298M [00:00<?, ?B/s]

In [ ]:
# size = 8000

# with open('data/sample.txt') as f:
#   l = f.readlines()

# pairs = []
# for i in l:
#   pairs.append(i.strip().split('\t')[:2])

# train_sources, train_targets = [i[1] for i in pairs[:size]], [i[0] for i in pairs[:size]]
# val_sources, val_targets = [i[1] for i in pairs[size:]], [i[0] for i in pairs[size:]]


In [7]:
with open('data/en_es_sentences.csv') as f:
  l = f.readlines()

print(l)

['Open your eyes.;Abra los ojos.\n', 'Point to where it hurts.;Señale dónde le duele.\n', 'Breath slowly.;Respire lentamente.\n', 'You need to exercise more.;Necesita hacer más ejercicio.\n', 'You need to do these tests.;Tiene que hacerse estos análisis.\n', 'Turn your head and cough.;Volte la cabeza y tosa.\n', 'Lift your head.;Levante la cabeza.\n', 'Open your mouth.;Abra la boca.\n', 'I’m going to examine your nose.;Voy a examinar su nariz.\n', '’m going to listen to your heart.;Voy a escuchar su corazón.\n', 'Please lie down.;Acuéstese por favor.\n', 'I have finished the exam.;Terminé la examinación.\n', 'I’m going to take your temperature.;Le voy a tomar su temperatura.\n', 'It’s time to take your medicine.;Es hora de tomar su medicina.\n', 'Please let me know if you feel any pain.;Por favor dígame si siente algún dolor.\n', 'Use the call light if you need help.;Oprima el botón si necesita ayuda.\n', 'I exercise every day.;Hago ejercicio todos los días.\n', 'My back hurts.;Me duel

In [8]:
size = 40

with open('data/en_es_sentences.csv') as f:
  l = f.readlines()

pairs = []
for i in l:
  pairs.append(i.strip().split(';'))


train_sources, train_targets = [i[0] for i in pairs[:size]], [i[1] for i in pairs[:size]]
val_sources, val_targets = [i[0] for i in pairs[size:]], [i[1] for i in pairs[size:]]

In [ ]:
# from itertools import cycle, islice

In [9]:
class StreamData(torch.utils.data.IterableDataset):

  def __init__(self, file_path, tokenizer, max_length = 100):
    self.file_path = file_path
    self.tokenizer = tokenizer
    self.max_token_length = max_length

  def parse_file(self, file_path):
    with open(file_path, 'r') as f:
      for line in f:
        source = line.split(';')[0] #change to \t at some point
        target = line.split(';')[1] #change to \t at some point
        self.model_inputs = self.tokenizer(source, max_length = self.max_token_length, truncation = True, padding = True)
        
        with tokenizer.as_target_tokenizer():
          self.labels = self.tokenizer(target, max_length= self.max_token_length, truncation = True, padding = True)
        self.model_inputs['labels'] = self.labels['input_ids']
        # yield source, target
        yield self.model_inputs


  def __iter__(self):
    return self.parse_file(self.file_path)
    # return self.get_stream(self.file_path)



^^ I believe the way this works is that instead of just working its way through the training set for a specified number of epochs, it will stream a batch of samples at a time, and it will continue to loop through the sample dataset in those batches for a specified number of steps.

E.g. if we have a training file with 40 lines and specify a batch size of 8, max_steps of 6, it will go through 5 batches of 8, then it will train on batch 1 again, then stop.

In [10]:
TRAIN_DATASET = StreamData('data/UFAL/medical_corpus_split/corpus.aa', tokenizer)
VAL_DATASET = StreamData('data/UFAL/medical_corpus_split/corpus.ab', tokenizer)

BATCH_SIZE = 8
TRAINING_STEPS = 10

In [27]:
training_lines = !wc -l data/UFAL/medical_corpus_split/corpus.aa
training_lines = int(training_lines[0].split(" ")[0])


print(f'Number of lines in Training Set: {training_lines}')

BATCH_SIZE = 8
EPOCHS = 1
TRAINING_STEPS = training_lines//BATCH_SIZE

Number of lines in Training Set: 632732


In [36]:
model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_language}-to-{target_language}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size= BATCH_SIZE,
    per_device_eval_batch_size= BATCH_SIZE,
    weight_decay=0.01,
    save_total_limit=3,
    save_steps = 79,
    num_train_epochs=1,
    predict_with_generate=True,
    max_steps = TRAINING_STEPS
)

#https://huggingface.co/docs/transformers/main_classes/trainer

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [37]:
f"{model_name}-finetuned-{source_language}-to-{target_language}"

'opus-mt-en-es-finetuned-en-to-es'

In [38]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
metric = load_metric("sacrebleu")

In [39]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [40]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=TRAIN_DATASET,
    eval_dataset=VAL_DATASET,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


max_steps is given, it will override any value given in num_train_epochs


In [42]:
trainer.train()

In [ ]:
### Now Test

In [ ]:
LATEST_CHECKPOINT = 10

tuned_model_name = f'opus-mt-en-es-finetuned-en-to-es/checkpoint-{LATEST_CHECKPOINT}'
tokenizer = MarianTokenizer.from_pretrained(tuned_model_name)
model = MarianMTModel.from_pretrained(tuned_model_name)

Didn't find file opus-mt-en-es-finetuned-en-to-es/checkpoint-10/target_vocab.json. We won't load it.
Didn't find file opus-mt-en-es-finetuned-en-to-es/checkpoint-10/added_tokens.json. We won't load it.
loading file opus-mt-en-es-finetuned-en-to-es/checkpoint-10/source.spm
loading file opus-mt-en-es-finetuned-en-to-es/checkpoint-10/target.spm
loading file opus-mt-en-es-finetuned-en-to-es/checkpoint-10/vocab.json
loading file None
loading file opus-mt-en-es-finetuned-en-to-es/checkpoint-10/tokenizer_config.json
loading file None
loading file opus-mt-en-es-finetuned-en-to-es/checkpoint-10/special_tokens_map.json
/usr/local/lib/python3.7/dist-packages/transformers/models/marian/tokenization_marian.py:198: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
loading configuration file opus-mt-en-es-finetuned-en-to-es/checkpoint-10/config.json
Model config MarianConfig {
  "_name_or_path": "Helsinki-NLP/opus-mt-en-es",
  "activation_dropou

In [ ]:
src_txt = ['Of these, 51% had generalised peritonitis at baseline.']

#correct translation: De estos, el 51% presentaba peritonitis generalizada en el momento basal.

In [ ]:
translated = model.generate(**tokenizer(src_txt, return_tensors="pt", padding=True))
[tokenizer.decode(t, skip_special_tokens=True) for t in translated]

['De estos, el 51% tenía peritonitis generalizada al inicio.']